In [41]:
import qiskit as qt
import numpy as np
from qiskit import Aer
import networkx as nx

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector


from qiskit.opflow import PauliExpectation, CircuitSampler, StateFn, CircuitStateFn
from qiskit.utils import QuantumInstance


## ADAPT QAOA (proposed in [Zhu et al](https://journals.aps.org/prresearch/pdf/10.1103/PhysRevResearch.4.033029))

Instead of having the same mixer operator for all layers, we choose the mixers from a pool of operators P = $\{A_j\}_{j\in Q}$. The algorithm proceeds as follows:
1. Start initially with $|\psi_0\rangle = |0^{\otimes n}\rangle$
2. To choose the (n+1)-th layer of QAOA, we first take create the state, $|\tilde{\psi}_n\rangle = \exp(-i\gamma_0 H_C)|\psi_n\rangle$, where $\gamma_0$ is a fixed number ($\gamma_0 = 0.01$ in the paper).
3. We compute the derivative of cost hamiltonian expectation value of this state for each operator in the pool. The j-th derivative is given by,
$$
-i \langle \tilde{\psi}_n| [H_C,A_j] | \tilde{\psi}_n\rangle
$$
4. If norm of the total gradient is below some chosen threshold **tol**, we stop the algorithm
5. Otherwise, we choose the operator corresponding to the largest component of the gradient as the next mixer layer and create the state,
$$
|\psi_{n+1}\rangle = \exp(-i\beta_{n+1}A_j)\exp(-i\gamma_{n+1} H_C)|\psi_{n}\rangle
$$
6. We optimize all the 2(n+1) parameters to minimize the cost hamiltonian
7. Go back to step 2

## Functions for QUBO evaluation

1. **qubo()** --> To get value of qubo function corresponding to an input state of binary variables
2. **compute_expectation()** --> Gets expectation value of qubo/cost hamiltonian from quantum measurement counts
3. **get_expectation_QAOA()** --> Gets count data from qiskit backend and calculates expectation value of qubo using compute_expectation(). This is the function getting minimized during optimization.

In [42]:
def qubo(qubo,state):
    """
    Returns value of qubo function corresponding to input state
    Args:
        qubo: QUBO object corresponding to the problem
        state: string denoting the state on which qubo is being evaluated
        
    Returns:
         Qubo function value for the state   
    
    """
    def bin2ising(bin_var):
        if int(bin_var) == 0:
            return 1
        else:
            return -1
    
    y = 0 
    for index, term in enumerate(qubo.terms):
        if len(term) == 1:
            y += bin2ising(state[term[0]]) * qubo.weights[index]
        
        elif len(term) == 2:
            y+= bin2ising(state[term[0]]) * bin2ising(state[term[1]]) * qubo.weights[index]
        
    return y


def compute_expectation(qubo_prob,counts,shots):
    """
    Calculates expectation value of a QUBO problem corresponding to circuit measurement
    Args:
        qubo_prob: QUBO object corresponding to the problem 
        counts: Dictionary of counts from qiskit measurement
        shots: total shots used for measurement
        
    Return:
        Expectation value of the QUBO for given measurement
    
    """
    exp = 0
    for state in counts:
        p = counts[state]/shots
        y = qubo(qubo_prob,state)
        exp += p * y
    return exp

from qiskit.compiler import transpile
def get_expectation_QAOA(theta_val,qubo_prob, qc, parameter, shots=1024):
    """
    Computes expectation of QUBO function corresponding to given QAOA params
    
    Args:
        param: parameters gamma's and beta's. First p values correspond to gamma parameters,
               last p values correspond to beta parameters
        shots: No. of shots used in measurement
    
    Returns:
        Expectation value of QUBO fucntion for the given QAOA params
        
    """

    qc_bind = qc.bind_parameters({parameter: theta_val})
    backend = Aer.get_backend('aer_simulator')

    qc_bind1 = transpile(qc_bind, backend=backend, seed_transpiler=11)
    qc_bind1.measure_all()

    counts = backend.run(qc_bind1, seed_simulator=10,
                         shots=shots).result().get_counts()

    return compute_expectation(qubo_prob,counts,shots)

## QAOA Cost Hamiltonian, Mixer Functions

1. **qaoa_hamil()** --> Adds the cost unitary to the circuit
2. **qaoa_mixer()** --> Adds the mixer unitary to the circuit
3. **qaoa_circuit_add()** --> Adds both the cost and mixer unitary by using both the previous functions


In [43]:
def qaoa_hamil(qc,qubits, qubo, gamma):
    """
    Creates a cost hamiltonian based on qubo
    
    Args:
        qc: QuantumCircuit on which cost hamiltonian is applied
        qubits: No. of qubits in circuit
        qubo: QUBO object corresponding to the problem
        gamma: Optimization parameter corresponding to cost hamiltonian for the layer
    """
    
    ising = qubo.terms
    weights = qubo.weights
    

    for index,term in enumerate(ising):
        if len(term)==1:
            qc.rz(2*gamma*weights[index],term[0])
            
        elif len(term)==2:
            qc.rzz(2*gamma*weights[index],term[0],term[1])
    


from qiskit.extensions import HamiltonianGate,UnitaryGate
def qaoa_mixer(qc,qubits,mixer,beta):
    """
    Creates a mixer hamiltonian based on chosen mixer
    
    Args:
        qc: QuantumCircuit on which mixer hamiltonian is applied
        qubits: No. of qubits in circuit
        mixer: mixer operator for the layer
        beta: Optimization parameter corresponding to mixer hamiltonian for the layer
    """
    qc.append(HamiltonianGate(mixer, beta),range(qubits))



def qaoa_circuit_add(qc,qubits,qubo,mixer,theta):
    """
    Adds both cost and mixer hamiltonian for the layer
    
    Args:
        qc: QuantumCircuit on which mixer hamiltonian is applied
        qubits: No. of qubits in circuit
        qubo: QUBO object corresponding to the problem
        mixer: mixer operator for the layer
        theta: Optimization parameters (gamma,beta) corresponding to mixer hamiltonian for the layer
    """
    
    qaoa_hamil(qc,qubits, qubo, theta[-2])
    qaoa_mixer(qc,qubits,mixer,theta[-1])
    


## Creating operator pool
First two function are just used to convert strings to corresponding pauli operators
1. **mixer_pool_single()** --> Returns a list of single qubit mixer operators
2. **mixer_pool_multi()** --> Returns a list of multi qubit mixer operators

In [44]:
## Helper functions to convert strings to corresponding Pauli operator

from qiskit.opflow import I, X, Y, Z
def str2Pauli(op_str):
    if op_str == "X":
        return X
    elif op_str == "Y":
        return Y
    elif op_str == "Z":
        return Z
    else:
        return I
    
def str_2_multiop(multiop_str):
    A = str2Pauli(multiop_str[0])
    for j in range(1,len(multiop_str)):
        A = A^str2Pauli(multiop_str[j])
    return A
    
    
    
def mixer_pool_single(qubits):
    """
    Single qubit operator pool
    Args:
        qubits: No. of qubits in circuit
    Returns:
        List of operators for single qubit operator pool
    
    """
    pool = []
    qaoa_x = []
    qaoa_y = []
    
    for i in range(qubits):
        op = ''
        for j in range(qubits):
            if i!=j:
                op = op + "I"
            else:
                op = op + 'X'
        pool.append([op])
        qaoa_x.append(op)
    pool.append(qaoa_x)
    
    for i in range(qubits):
        op = ''
        for j in range(qubits):
            if i!=j:
                op = op + "I"
            else:
                op = op + 'Y'
        pool.append([op])
        qaoa_y.append(op)
    pool.append(qaoa_x)
        
    pool_ops = []
    
    for ops in pool:
        if len(ops) == 1:
            pool_ops.append(str_2_multiop(ops[0]))
        else:
            pauli_op = str_2_multiop(ops[0])
            for i in range(1,len(ops)):
                pauli_op += str_2_multiop(ops[i])
            pool_ops.append(pauli_op)
    return pool_ops

def mixer_pool_multi(qubits):
    """
    Gives a multi qubit operator pool
    Args:
        qubits: No. of qubits in circuit
    Returns:
        List of operators for multi qubit operator pool
    
    """
    paulis = ["X","Y","Z"]
    
    pool = []
    for i in range(qubits-1):
        for j in range(i+1,qubits):
            for pauli_op1 in paulis:
                for pauli_op2 in paulis:
                    op = 'I'*i + pauli_op1 + 'I'*(j-i-1) +pauli_op2 + 'I'*(qubits-j-1)
                    pool.append(op)
    
    pool_op = []
    for op in pool:
        pool_op.append(str_2_multiop(op))
                    
    return pool_op


## Evaluating derivative
The below functions are used to evaluate derivative for each mixer operator by evaluating the commutator,
$$
-i \langle \tilde{\psi}_n| [H_C,A_j] | \tilde{\psi}_n\rangle
$$

In [58]:
## Helper functions for converting Hamiltonian to Pauli operators
def hamil_terms2Pauli(qubits,pos):
    A = ''
    for i in range(qubits):
        if i in pos:
            A = A + 'Z'
        else:
            A = A + 'I'
            
    return str_2_multiop(A)

def hamil_Pauli(qubo,qubits):
    ising = qubo.terms
    weights = qubo.weights  
    
    hamil_op = str_2_multiop("I"*qubits)
    
    for index,term in enumerate(ising):
            hamil_op += weights[index] * hamil_terms2Pauli(qubits,term)
    
    return hamil_op



from qiskit import Aer
from qiskit.opflow import commutator
from qiskit.opflow import PauliOp

def derivative(hamil,mixer,qc):
    """
    Evaluates derivative using commutator method
    Args:
        hamil: Pauli operator form of cost Hamiltonian
        mixer: Pauli operator form of mixer operator
        qc: QuantumCircuit on which commutator is being evaluated
    Returns:
        Derivative wrt to mixer operator
    """
    
    ## Setting up backend for measurement
    backend = Aer.get_backend( 'aer_simulator' )
    shots = 1024
    qc1 = transpile(qc, backend=backend, seed_transpiler=11)
    
    ## State after circuit implementation on which commutator will be evaluated
    psi = CircuitStateFn(qc1)
    
    q_instance = QuantumInstance(backend, shots = shots)
    
    ## Commutator is evaluated
    deriv_op = (-1j) * commutator(hamil,mixer)
    
    ## Expectation value of each term of commutator is evaluated seperately
    deriv = 0
    
    ## Checking if commutator is zero
    if type(deriv_op) == PauliOp:
        return deriv
    
    else:
        for op in deriv_op:
            coefficient = op.coeffs
            pauli_str = PauliOp(op.primitive.paulis[0])
            
            if coefficient != 0 :
                measurable_expression = StateFn(pauli_str, is_measurement = True).compose(psi)
                expectation = PauliExpectation().convert(measurable_expression)

                pauli_eval = CircuitSampler(q_instance).convert(expectation).eval()
                deriv += pauli_eval * coefficient

        return complex(deriv).real



## Optimization using scipy

In [48]:
from scipy.optimize import minimize
def initialize_param(theta):
    """
    Initializes the theta value by adding addditional random gamma,beta at the end
    
    """
    
    theta = list(theta)
    random_guess_gamma = np.random.uniform(low = 0, high = 2*np.pi)
    random_guess_beta = np.random.uniform(low = 0, high = 2*np.pi)
    
    theta.append(random_guess_gamma)
    theta.append(random_guess_beta)    
    
    return theta
    

def optimize_params(qubo_prob,circ,parameter,theta_init):
    """
    Optimizes parameters of qaoa using scipy.optimize.minimize
    
    Args:
        qubo_prob: QUBO object corresponding to problem
        circ: QuantumCircuit which is being optimized wrt gamma,betas
        parameter: ParameterVector for storing optimizing paramaeters in the circuit
        theta_init: Variational parameters being optimized in the circuit
    Returns:
        Optimized parameters and corresponiding minimum qubo function value
    """
    
    optimizer = "COBYLA"
    theta_new = initialize_param(theta_init)
    res_adapt_qaoa_min = minimize(get_expectation_QAOA,theta_new,method = optimizer,args = (qubo_prob,circ,parameter))
    
    minimum = res_adapt_qaoa_min.fun
    
    ## I ran the algorithm multiple times with random initialization and choose the the one with minimum 
    ## function value to avoid local minimas. Not sure what's an efficient way of doing this
    for _ in range(2):
        theta_new = initialize_param(theta_init)
        res_adapt_qaoa = minimize(get_expectation_QAOA,theta_new,method = optimizer,args = (qubo_prob,circ,parameter))
        
        if res_adapt_qaoa.fun < minimum:
            res_adapt_qaoa_min = res_adapt_qaoa
        
    return res_adapt_qaoa_min.x,minimum

## Full Adapt QAOA Function

In [70]:
def adapt_qaoa(qubits,qubo, mixer_pool, tol = 0.015):
    """
    Runs the full adapt QAOA algorithm until gradient norm decreases below tolerance
    Args:
        qubits: No. of qubits used in circuit
        qubo: QUBO object corresponding to the problem
        mixer_pool: List of mixers from which optimal mixer is chosen each layer
        tol: Tolerance of gradient norm upto which we run the algorithm
    Return:
        QuantumCircuit with all the mixer layers 
        ParameterVector for storing optimizing parameters of circuit
        List of optimal parameters for the circuit
        List of gradient norms for each layer       
    """
    qc = QuantumCircuit(qubits)
    qc.h(range(qubits))
    
    qc.barrier()
    p = 0 
    theta = []
    params_circ = ParameterVector("theta", 2)
    
    gamma_0 = 0.01
    mixers_used = []
    gradient_layer = []
    while True:
        
        gradients = []
        qubo_min_layer = []
        if len(theta)<2:
            qc_grad = qc.copy()
        else:    
            qc_grad = qc.bind_parameters({params_circ: theta})

        qaoa_hamil(qc_grad,qubits,qubo,gamma_0)
        
        
        ## Measures gradient wrt to operator pool
        for mixer in mixer_pool:
            gradients.append(derivative(hamil_Pauli(qubo,qubits),mixer,qc_grad))
        
        ## Stores scaled gradient norm for each layer
        scaled_gradient_norm = np.linalg.norm(gradients)/len(pool)
        gradient_layer.append(scaled_gradient_norm)
        
        
        ## If gradient_norm is below tolerance we end the algorithm
        ## For large pools we are scaling the tolerance with the size of the pool
        if scaled_gradient_norm < tol:
            print(f"QAOA converged with gradient norm = {scaled_gradient_norm}")
            return qc,params_circ,theta
        
        ## Update QAOA layer number
        p+=1
        
        ##Stores the mixers being used
        mixers_used.append(mixer_pool[np.argmax(np.abs(gradients))])
        
        ## Resize parameter vector to accomodate more variables
        params_circ.resize(2*p)
        
        ## Adds new layer of QAOA
        qaoa_circuit_add(qc,qubits,qubo,mixers_used[-1],params_circ)
        qc.barrier()
        
        ## Optimize for the new layer
        theta,qubo_min = optimize_params(qubo,qc,params_circ, theta)
        
        qubo_min_layer.append(qubo_min)
        
        print(f"Layer = {p}, Scaled Gradient Norm = {scaled_gradient_norm}, Mixer = {mixers_used[-1]} ,Miminized QUBO = {qubo_min}")

## Example problem: MaxCut


In [89]:
nodes = 5
G = nx.generators.random_regular_graph(n=nodes, d=2, seed=42)
for (u, v) in G.edges():
    G.edges[u,v]['weight'] = np.random.uniform()
    
from openqaoa.problems import MaximumCut   
maxcut_qubo = MaximumCut(G).qubo

### Exact Solution

In [90]:
hamiltonian = maxcut_qubo.hamiltonian

# import the brute-force solver to obtain exact solution
from openqaoa.utilities import ground_state_hamiltonian
energy, configuration = ground_state_hamiltonian(hamiltonian)

print(f"Ground State energy: {energy}, Solution: {configuration}")

Ground State energy: -2.2991871036798592, Solution: ['01110', '10001']


Solving using the Adapt QAOA method

In [91]:
pool = mixer_pool_single(nodes) + mixer_pool_multi(nodes)

qc,parameter,theta = adapt_qaoa(nodes,maxcut_qubo,pool)

Layer = 1, Scaled Gradient Norm = 0.03718205791415013, Mixer = IIYIZ ,Miminized QUBO = -0.18255760866034032
Layer = 2, Scaled Gradient Norm = 0.03627966543519053, Mixer = IYIIZ ,Miminized QUBO = -0.6856260809434812
Layer = 3, Scaled Gradient Norm = 0.02715046535696942, Mixer = 1.0 * XIIII
+ 1.0 * IXIII
+ 1.0 * IIXII
+ 1.0 * IIIXI
+ 1.0 * IIIIX ,Miminized QUBO = -1.5504523688615246
Layer = 4, Scaled Gradient Norm = 0.017335013824390528, Mixer = IIIXI ,Miminized QUBO = -1.9449342805996803
QAOA converged with gradient norm = 0.011977338829772607


Running the optimized circuit to get maximum probable states as solution. For maxcut there will always be two solutions since the nodes within each cut can be labelled either as zero or one

In [92]:
qc_bind = qc.bind_parameters({parameter:theta})
backend = Aer.get_backend('aer_simulator')

qc_bind1 = transpile(qc_bind, backend=backend, seed_transpiler=11)
qc_bind1.measure_all()

counts = backend.run(qc_bind1, seed_simulator=10, shots=1024).result().get_counts()

In [117]:
value_key_pairs = ((value/1024, key) for (key,value) in counts.items())
sorted_value_key_pairs = sorted(value_key_pairs, reverse=True)

ordered_counts = {k: v for v, k in sorted_value_key_pairs[:1]}
print(f"Most likely state from Adapt QAOA: {ordered_counts}")


Most likely state from Adapt QAOA: {'10001': 0.248046875}


## Comparison with solution from OpenQAOA

In [108]:
from openqaoa.backends import create_device
from openqaoa import QAOA

q = QAOA()

# device
device = create_device(location='local', name='qiskit.qasm_simulator')
q.set_device(device)

# circuit properties
q.set_circuit_properties(p=4, param_type='standard', init_type='rand', mixer_hamiltonian='x')


q.set_backend_properties(n_shots = 1024)
q.set_classical_optimizer(method='COBYLA', maxiter=200, tol=0.001, cost_progress=True, parameter_log=True)

q.compile(maxcut_qubo)
q.optimize()

In [109]:
q.result.most_probable_states

{'solutions_bitstrings': ['10001'], 'bitstring_energy': -2.2991871036798592}

In [113]:
q.result.optimized['cost']

-1.716265260358

In [116]:
counts_qaoa = q.result.optimized['measurement_outcomes']

value_key_pairs = ((value/1024, key) for (key,value) in counts_qaoa.items())
sorted_value_key_pairs = sorted(value_key_pairs, reverse=True)

ordered_counts_qaoa = {k: v for v, k in sorted_value_key_pairs[:1]}
print(f"Most probable state from OpenQAOA: {ordered_counts_qaoa}")

Most probable state from OpenQAOA: {'10001': 0.2373046875}
